# Notebook to synch External Volumes
---

### Parameters:

Source Workspace : uri to source workspace

Destination Workspace : uri to destination workspace

Notebook, checks whether volume 'bronze' is existing in the source ws and then makes sure, that the volume is not existing in dest and then creates it



In [0]:
dbutils.widgets.text("source_workspace", "", "Source Workspace")
dbutils.widgets.text("destination_workspace", "", "Destination Workspace")

In [0]:
source_workspace = dbutils.widgets.get("source_workspace")
destination_workspace = dbutils.widgets.get("destination_workspace")

In [0]:
# variables
catalog_name = 'catadb360dev'
schema_name = 'schemaadb360dev'
volume_name = 'bronze'

In [0]:
# get the credentials for sp from a keyvault backed secret scope
clientid = dbutils.secrets.get('kvbacked', 'clientid')
clientsecret = dbutils.secrets.get('kvbacked', 'clientsecret')
tenantid = dbutils.secrets.get('kvbacked', 'tenantid')



In [0]:
# imports
from databricks.sdk import WorkspaceClient
from databricks.sdk.errors.platform import ResourceAlreadyExists
from databricks.sdk.service import catalog

In [0]:
sourceWs = WorkspaceClient(azure_client_id=clientid, azure_client_secret=clientsecret, azure_tenant_id=tenantid, host=source_workspace)
destWs = WorkspaceClient(azure_client_id=clientid, azure_client_secret=clientsecret, azure_tenant_id=tenantid, host=destination_workspace)

In [0]:
# if the volume exits in the source workspace
sourceVols = [x for x in sourceWs.volumes.list(catalog_name=catalog_name, schema_name=schema_name)]

assert len(sourceVols) == 1

In [0]:
# if the volume does not exist in destination
destVols = [x for x in destWs.volumes.list(catalog_name=catalog_name, schema_name=schema_name)]

assert len(destVols) == 0

In [0]:
# create the volume
try:
    destWs.volumes.create(
            catalog_name=catalog_name, 
            schema_name=schema_name, 
            name=volume_name, 
            comment='bronze volume',
            volume_type = catalog.VolumeType.EXTERNAL,
            storage_location=sourceVols[0].storage_location
        )
except ResourceAlreadyExists:
    print(f"volume {name} already exists, doing nothing")

In [0]:
print('done')